In [1]:
from utils.preprocess import JSONLoader
fields = ['business_id','categories','city']
# fields = None
city = ['Toronto']
categories_all =['Restaurants', 'Food', 'Sandwiches', 'Fast Food', 'American (Traditional)', 'Pizza',
                 'Italian', 'Burgers', 'Mexican','Breakfast & Brunch', 'American (New)', 'Chinese', 'Specialty Food', 'Bakeries', 'Desserts',
                 'Japanese', 'Ice Cream & Frozen Yogurt', 'Chicken Wings', 'Seafood', 'Salad', 'Sushi Bars',
                 'Asian Fusion', 'Thai', 'Indian']

business = 'business.json'
#data_dir = 'data/dataset'
data_dir = 'C:/Users/USer/social data mining/Project/dataset/'

jl = JSONLoader(business, data_dir, fields = fields)
jl.set_condition(city=city, categories=categories_all)
f_b, all_restaurants = jl.sample(1000000)
print(len(all_restaurants))
all_rest_id = set([restaurant[0] for restaurant in all_restaurants])
print(len(all_rest_id))

8627
8627


In [2]:
# Load the reviews
review = 'review.json'
#data_dir = 'data/dataset'
data_dir = 'C:/Users/USer/social data mining/Project/dataset/'

fields = ['user_id','business_id','stars','text' ]
jl = JSONLoader(review, data_dir, fields = fields)
jl.set_condition(business_id = all_rest_id)

f_, all_reviews = jl.sample(1000000)
len(all_reviews)

69192

In [3]:
categories =['Seafood', 'Burgers', 'Italian', 'Japanese', 'Chinese']

rest_id = {}

for category in categories:
    jl = JSONLoader(business, data_dir, fields = fields)
    jl.set_condition(city=city, categories=[category])
    f_b, categorized_restaurants = jl.sample(1000000)
#     print(len(categorized_restaurants))
    rest_id[category] = set([restaurant[0] for restaurant in categorized_restaurants])
#     print(len(rest_id[category]))

In [5]:
# Load the reviews
categorized_reviews = {}
review = 'review.json'
#data_dir = 'data/dataset'
data_dir = 'C:/Users/USer/social data mining/Project/dataset/'

fields = ['user_id','business_id','stars','text']

for category in categories:
    jl = JSONLoader(review, data_dir, fields = fields)
    jl.set_condition(business_id = rest_id[category])
    f_, categorized_reviews[category] = jl.sample(1000000)
    print(len(categorized_reviews[category]))

3278
3005
4874
5404
4164


In [174]:
# To raise the accuracy, we have to consider plural.

def toPlural(word):
    if word.endswith('y'):
        return word[:-1] + 'ies'
    elif word[-1] in 'sx' or word[-2:] in ['sh', 'ch']:
        return word + 'es'
    elif word.endswith('an'):
        return word[:-2] + 'en'
    else:
        return word + 's'

In [7]:
import re, string

def eliminatePunctuation(text):
    out = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return out

In [9]:
# For particular review texts
from nltk.corpus import stopwords

stopwords.words('english')
stopwords = stopwords.words('english')

categorized_review_texts = {}
categorized_texts_processed = {}
for category in categories:
    categorized_review_texts[category] = [review[3] for review in categorized_reviews[category]]
    categorized_texts_processed[category] = [[word for word in eliminatePunctuation(text_review).lower().split() if word not in stopwords] for text_review in categorized_review_texts[category]]

In [10]:
concatenated_categorized_texts_processed = {}
for category in categories:
    concatenated_categorized_texts_processed[category] = [sum(categorized_texts_processed[category], [])]

In [11]:
# For all review texts
from nltk.corpus import stopwords

all_text_reviews = [review[3] for review in all_reviews]

stopwords.words('english')
stopwords = stopwords.words('english')

texts = [[word for word in eliminatePunctuation(text_review).lower().split() if word not in stopwords] for text_review in all_text_reviews]

In [15]:
from gensim import corpora

all_dictionary = corpora.Dictionary(texts)
corpus = [all_dictionary.doc2bow(text) for text in texts]

In [16]:
'''
testing, please just skip this part.
'''
print(all_dictionary[7])  # just for test
print(all_dictionary.token2id['would']) # just for test

would
7


In [17]:
import gensim.models as gm
tfidf = gm.TfidfModel(corpus)

In [19]:
# TF-IDF per review text per category

categorized_corpus = {}
categorized_tfidf = {}

for category in categories:
    categorized_corpus[category] = [all_dictionary.doc2bow(text) for text in categorized_texts_processed[category]]
    categorized_tfidf[category] = tfidf[categorized_corpus[category]]
    print("# of review text under ", category, "= ", len(categorized_tfidf[category]))

# of review text under  Seafood =  3278
# of review text under  Burgers =  3005
# of review text under  Italian =  4874
# of review text under  Japanese =  5404
# of review text under  Chinese =  4164


In [20]:
# TF-IDF per category by being concatenated

concatenated_categorized_corpus = {}
concatenated_categorized_tfidf = {}

for category in categories:
    concatenated_categorized_corpus[category] = [all_dictionary.doc2bow(text) for text in concatenated_categorized_texts_processed[category]]
    concatenated_categorized_tfidf[category] = tfidf[concatenated_categorized_corpus[category]]

In [21]:
# By category, display the words according to tf-idf

import operator

tfidf_under_category = {}
words_sorted_by_tfidf = {}
just = {}
for category in categories:
    intermediate = {}
    for entry in concatenated_categorized_tfidf[category][0]:
        intermediate[all_dictionary[entry[0]]] = entry[1]
    words_sorted_by_tfidf[category] = [ word for (word, value) in sorted(intermediate.items(), key=operator.itemgetter(1), reverse=True)]
    print(category, words_sorted_by_tfidf[category][:20])

Seafood ['lobster', 'oysters', 'seafood', 'crab', 'us', 'fish', 'restaurant', 'good', 'service', 'food', 'came', 'fried', 'table', 'great', 'would', 'oyster', 'ordered', 'one', 'time', 'menu']
Burgers ['burger', 'burgers', 'fries', 'chicken', 'poutine', 'patty', 'bacon', 'good', 'like', 'place', 'cheese', 'order', 'toppings', 'rings', 'really', 'bun', 'onion', 'get', 'time', 'one']
Italian ['pizza', 'pasta', 'italian', 'restaurant', 'us', 'good', 'food', 'great', 'service', 'sauce', 'would', 'wine', 'really', 'nice', 'like', 'one', 'ordered', 'place', 'salad', 'bread']
Japanese ['sushi', 'ramen', 'japanese', 'sashimi', 'roll', 'rolls', 'salmon', 'restaurant', 'good', 'place', 'food', 'lunch', 'service', 'tempura', 'really', 'like', 'pork', 'noodles', 'soup', 'rice']
Chinese ['chinese', 'sum', 'dim', 'lobster', 'fried', 'noodles', 'dumplings', 'rice', 'soup', 'noodle', 'pork', 'food', 'dishes', 'restaurant', 'chicken', 'dish', 'good', 'place', 'order', 'like']
